# Installation

In [ ]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

In [ ]:
!pip3 install llama-index --upgrade --no-cache-dir --force-reinstall

In [ ]:
!pip install llama-index-llms-gradient

In [ ]:
!pip install llama-index-embeddings-gradient

In [17]:
!pip install gradientai --quiet

In [44]:
import os
import json
# from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = 'CK1zDOU4BQ03NoMJMUDndWM8oAoNFMpm'
os.environ['GRADIENT_WORKSPACE_ID'] =  "55708ca1-7b2b-42b1-ae1b-7d75ef4bc8c7_workspace"


from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index.core import ServiceContext
from llama_index.core import set_global_service_context
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings.gradient import GradientEmbedding
from llama_index.llms.gradient import GradientBaseModelLLM
# from llama_index.vector_stores import CassandraVectorStore

# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
import json

cloud_config= {
  'secure_connect_bundle': 'secure-connect-nlp.zip'
}

# This token json file is autogenerated when you donwload your token,
# if yours is different update the file name below
with open("premkumarc1111@gmail.com-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")



llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)


embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

documents = SimpleDirectoryReader("docs").load_data()
print(f"Loaded {len(documents)} document(s).")

index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()

response = query_engine.query("what is the payment history")
print(response)



# Import OS & JSON Modules

In [45]:
import os
import json
# from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = 'CK1zDOU4BQ03NoMJMUDndWM8oAoNFMpm'
os.environ['GRADIENT_WORKSPACE_ID'] =  "55708ca1-7b2b-42b1-ae1b-7d75ef4bc8c7_workspace"

# Import Cassandra & llama Index

In [21]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index.core import ServiceContext
from llama_index.core import set_global_service_context
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings.gradient import GradientEmbedding
from llama_index.llms.gradient import GradientBaseModelLLM
# from llama_index.vector_stores import CassandraVectorStore

In [ ]:
import cassandra
print (cassandra.__version__)

# Connect to the VectorDB

In [23]:
# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
import json

cloud_config= {
  'secure_connect_bundle': 'secure-connect-nlp.zip'
}

# This token json file is autogenerated when you donwload your token,
# if yours is different update the file name below
with open("premkumarc1111@gmail.com-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.11-09ec37c912ed


# Define the Gradient's Model Adapter for LLAMA-2

In [26]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

# Configure Gradient embeddings

In [27]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

In [38]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

C:\Users\PRKUMAR\AppData\Local\Temp\ipykernel_20580\3337229202.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


# Load the PDFs

In [39]:
documents = SimpleDirectoryReader("docs").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 2 document(s).


# Setup and Query Index

In [40]:
index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()

In [ ]:
# Import VectorStoreIndex class

# Define batch size
batch_size = 50  # Adjust as needed based on your system resources and processing time



# Determine the total number of batches
total_batches = (len(documents) + batch_size - 1) // batch_size

# Initialize an empty list to store batch indexes
batch_indexes = []

# Process documents in batches
for i in range(total_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, len(documents))
    batch_documents = documents[start_index:end_index]

    # Create VectorStoreIndex from batch documents
    batch_index = VectorStoreIndex.from_documents(batch_documents, service_context=service_context)
    batch_indexes.append(batch_index)

# Merge batch indexes into a single index
merged_index = batch_indexes[0]  # Initialize with the first batch index

for index in batch_indexes[1:]:
    merged_index = merged_index.merge(index)

# Create query engine from merged index
query_engine = merged_index.as_query_engine()

In [43]:
response = query_engine.query("what is the payment history")
print(response)

 Based on the context information provided, the payment history for the user is as follows:

* 29-FEB: 631.60
* 31-JAN: 706.82
* 31-DEC: 706.82

Therefore, the payment history for the user shows three payments made in February, January, and December, respectively, with the amounts being 631.60, 706.82, and 706.82, respectively.
